<a href="https://colab.research.google.com/github/dannem1337/Bart-Bartender/blob/main/Random_forest_for_emotion_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
import cv2
import matplotlib.pyplot as plt
import os
warnings.filterwarnings("ignore", module="feat")
import torch as th
from google.colab import files
import pandas as pd
import numpy as np




In [26]:
uploaded = files.upload()

Saving aus000800.csv to aus000800 (1).csv


In [5]:
dataset_df= pd.read_csv("aus000800.csv")
dataset_df = dataset_df.dropna()
dataset_df.head()

,emotion,AU01,AU02,AU04,AU05,AU06,AU07,AU09,AU10,AU11,...,AU14,AU15,AU17,AU20,AU23,AU24,AU25,AU26,AU28,AU43
0,neutral,0.450774,0.289915,0.409713,0.518726,0.086218,0.0,0.187309,0.354838,0.0,...,0.320690,0.411641,0.431646,0.0,0.277122,0.335435,0.262999,0.189863,0.051967,0.051370
1,disgust,0.500450,0.314694,0.625174,0.335747,0.262984,0.0,0.504238,0.383201,0.0,...,0.544159,0.440429,0.495913,0.0,0.514737,0.420401,0.052358,0.143576,0.500994,0.155117
2,sad,0.273191,0.191327,0.140938,0.358091,0.246593,0.0,0.312881,0.188845,1.0,...,0.284598,0.761539,0.491468,0.0,0.134049,0.670237,0.024796,0.109462,0.325429,0.191367
3,neutral,0.464508,0.301702,0.500370,0.296161,0.189114,0.0,0.521304,0.039475,0.0,...,0.491734,0.163430,0.552469,0.0,0.419418,0.306920,0.224105,0.072518,0.652248,0.505568
4,happy,0.274483,0.232007,0.601821,0.281365,0.900241,1.0,0.784789,0.198816,0.0,...,0.703261,0.549239,0.425561,0.0,0.203916,0.561599,0.966706,0.108249,0.464104,0.786888


In [7]:
# Calculate the distribution of the 'emotion' column
emotion_counts = dataset_df['emotion'].value_counts()

# Calculate the percentage distribution
emotion_percentages = (emotion_counts / emotion_counts.sum()) * 100

# Display the percentage distribution
print(emotion_percentages)

emotion
neutral     20.973269
happy       20.950423
surprise    16.860864
sad         11.948823
fear        11.743203
angry       10.418095
disgust      7.105323
Name: count, dtype: float64


In [9]:
from sklearn.model_selection import train_test_split

# Assuming your dataset has already been loaded into 'dataset_df'
X = dataset_df.drop(columns=["emotion"])  # Features
y = dataset_df["emotion"]  # Target variable

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [10]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier

# Apply SMOTE to oversample the minority class
smote = SMOTE(sampling_strategy="auto", random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Train the Random Forest on the resampled data
model = RandomForestClassifier(random_state=42)
model.fit(X_train_balanced, y_train_balanced)


RandomForestClassifier(random_state=42)

In [11]:
from sklearn.metrics import classification_report, accuracy_score

# Predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Detailed classification report
print(classification_report(y_test, y_pred))

Accuracy: 45.66%
              precision    recall  f1-score   support

       angry       0.39      0.30      0.34        91
     disgust       0.27      0.27      0.27        62
        fear       0.23      0.34      0.27       103
       happy       0.77      0.70      0.73       183
     neutral       0.41      0.52      0.46       184
         sad       0.28      0.20      0.23       105
    surprise       0.64      0.52      0.57       148

    accuracy                           0.46       876
   macro avg       0.43      0.41      0.41       876
weighted avg       0.48      0.46      0.46       876



# Make a more thorough search for the best model


In [12]:
#This code takes 1h 40 minutes to run.

from sklearn.model_selection import GridSearchCV

# Hyperparameter grid to search
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],    # Minimum number of samples required at each leaf node
    'max_features': ['sqrt', 'log2'],  # Number of features to consider when looking for the best split
    'bootstrap': [True, False]  # Whether bootstrap samples are used when building trees
}

forestmodel= RandomForestClassifier(random_state=42)
# Perform GridSearchCV using the training and validation sets (cross-validation will use the validation data)
grid_search = GridSearchCV(estimator=forestmodel, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_balanced, y_train_balanced)

# Print the best hyperparameters and cross-validation score
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Cross-Validation Score: {:.2f}".format(grid_search.best_score_))

# Once the best model is found, use it to predict on the test set
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model on the test set
print("Test Set Evaluation:")
print(classification_report(y_test, y_pred))
print("Best Hyperparameters:", grid_search.best_params_)


accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Fitting 5 folds for each of 432 candidates, totalling 2160 fits


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Hyperparameters: {'bootstrap': False, 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best Cross-Validation Score: 0.65
Test Set Evaluation:
              precision    recall  f1-score   support

       angry       0.39      0.26      0.32        91
     disgust       0.37      0.29      0.32        62
        fear       0.22      0.31      0.25       103
       happy       0.78      0.70      0.74       183
     neutral       0.42      0.59      0.49       184
         sad       0.28      0.19      0.23       105
    surprise       0.62      0.51      0.56       148

    accuracy                           0.46       876
   macro avg       0.44      0.41      0.42       876
weighted avg       0.48      0.46      0.46       876

Test Accuracy: 46.35%


In [14]:
# Save teh model to my google drive
from google.colab import drive
import joblib

# Mount Google Drive
drive.mount('/content/drive')

# Define the path where you want

Mounted at /content/drive


In [15]:

model_path = '/content/drive/My Drive/Y_2_master/Intelligent interactive systems P2/4635_RF_emotion_model.pkl'  # Adjust the path as needed

# Save the best model to Google Drive
joblib.dump(best_model, model_path)

print(f"Model saved to {model_path}")


Model saved to /content/drive/My Drive/Y_2_master/Intelligent interactive systems P2/4635_RF_emotion_model.pkl


#Try again without SMOTE
Do NOT apply SMOTE as it made the perforamnce worse for other models we previously tried.

In [36]:
dataset_df= pd.read_csv("aus000800.csv")
dataset_df = dataset_df.dropna()
dataset_df.head()

from sklearn.model_selection import train_test_split

# Assuming your dataset has already been loaded into 'dataset_df'
X = dataset_df.drop(columns=["emotion"])  # Features
y = dataset_df["emotion"]  # Target variable

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [37]:
#This code takes 1h 40 minutes to run.

from sklearn.model_selection import GridSearchCV

# Hyperparameter grid to search
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],    # Minimum number of samples required at each leaf node
    'max_features': ['sqrt', 'log2'],  # Number of features to consider when looking for the best split
    'bootstrap': [True, False]  # Whether bootstrap samples are used when building trees
}

forestmodel2= RandomForestClassifier(random_state=42)
# Perform GridSearchCV using the training and validation sets (cross-validation will use the validation data)
grid_search = GridSearchCV(estimator=forestmodel2, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and cross-validation score
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Cross-Validation Score: {:.2f}".format(grid_search.best_score_))

# Once the best model is found, use it to predict on the test set
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model on the test set
print("Best Hyperparameters:", grid_search.best_params_)
print("Test Set Evaluation:")
print(classification_report(y_test, y_pred))

accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Fitting 5 folds for each of 432 candidates, totalling 2160 fits


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Hyperparameters: {'bootstrap': True, 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best Cross-Validation Score: 0.47
Best Hyperparameters: {'bootstrap': True, 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Test Set Evaluation:
              precision    recall  f1-score   support

       angry       0.55      0.25      0.35        91
     disgust       0.53      0.15      0.23        62
        fear       0.25      0.05      0.08       103
       happy       0.77      0.73      0.75       183
     neutral       0.38      0.71      0.49       184
         sad       0.17      0.22      0.19       105
    surprise       0.58      0.56      0.57       148

    accuracy                           0.46       876
   macro avg       0.46      0.38      0.38       876
weighted avg       0.48      0.46      0.44       876

Test Accuracy: 46.46%


In [38]:
# Save teh model to my google drive
from google.colab import drive
import joblib

# Mount Google Drive
drive.mount('/content/drive')

# Define the path where you want

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:

model_path = '/content/drive/My Drive/Y_2_master/Intelligent interactive systems P2/4646_NoSMOTE_RF_emotion_model.pkl'  # Adjust the path as needed

# Save the best model to Google Drive
joblib.dump(best_model, model_path)

print(f"Model saved to {model_path}")


Model saved to /content/drive/My Drive/Y_2_master/Intelligent interactive systems P2/4646_NoSMOTE_RF_emotion_model.pkl
